# Chapter 3. Deep Learning with PyTorch
## Creation of tensors

In [0]:
import torch
import numpy as np

In [0]:
a = torch.FloatTensor(3,2)

In [3]:
a

tensor([[2.6741e-36, 0.0000e+00],
        [3.7835e-44, 0.0000e+00],
        [       nan, 0.0000e+00]])

Pytorch에서 3*2 사이즈의 tensor를 만들었지만 초기화 하지 않았기 때문에 tensor를 clear하기 위해서 아래의 코드를 작성한다.

### 첫번째 방법 : Inplace

In [4]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

constructor로 tensor를 만드는 또 다른 방법은 Python iterable (list, tuple과 같은) 를 넣는 것이다. 이것은 새롭게 만들어지는 tensor의 내용으로 사용된다.

In [5]:
torch.FloatTensor([[1,2,3],[3,2,1]])

tensor([[1., 2., 3.],
        [3., 2., 1.]])

### 두번째 방법 : Functional

Numpy를 이용하여 똑같은 zero object를 만들어 본다.

In [0]:
n = np.zeros(shape=(3,2))

In [7]:
n

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [0]:
b = torch.tensor(n)

In [9]:
b

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)

이전 코드 예시에서는 numpy로 default인 zero initialize한 a double (64-bit float) array를 만들었다.

그래서 the DoubleTensor type을 가지게 되었다.

하지만 보통 DL에서는 double precision이 요구되지 않고 이것은 extra memory와 performance overhead를 추가한다.

보통 32-bit float type 또는 16-bit float type를 사용한다.

이러한 tensor들을 만들기 위해 dtype으로 세부사항을 명시해준다.

In [0]:
n = np.zeros(shape=(3,2), dtype=np.float32)

In [11]:
torch.tensor(n)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

또 다른 방법으로는 torch.tensor에도 dtype argument가 있기 때문에 이를 선택해도 된다.

하지만, 여기서 주의 사항은 이 dtype은 PyTorch type의 명시조건을 써줘야 한다. 

앞선 예제에서는 np.float32였지만 여기에서는 torch.float32이다.

PyTorch type들에 대한 것은 torch package에 있으며 예로는 torch.float32, torch.unit8 등이 있다.

In [0]:
n = np.zeros(shape=(3,2))

In [13]:
torch.tensor(n, dtype=torch.float32)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

(version 참고) 이전 버전에서는 torch.from_numpy()

# Scalar tensors

In [0]:
a = torch.tensor([1,2,3])

In [15]:
a

tensor([1, 2, 3])

In [0]:
s = a.sum()

In [17]:
s

tensor(6)

In [18]:
s.item()

6

In [19]:
torch.tensor(1)

tensor(1)

# GPU tensors

CPU tensor를 만든다음에 GPU memory에 복사하기

In [0]:
a = torch.FloatTensor([2,3])

In [21]:
a

tensor([2., 3.])

In [22]:
ca = a.cuda();ca

tensor([2., 3.], device='cuda:0')

a와 ca 둘다 연산에 쓰이고 모든 GPU-specific machinery는 사용자에게 보여진다.

In [23]:
a+1

tensor([3., 4.])

In [24]:
ca+1

tensor([3., 4.], device='cuda:0')

In [25]:
ca.device

device(type='cuda', index=0)

# Tensor and gradients

gradient-leaf machinery를 잘 이해하기 위한 코드 실습이다.

In [0]:
v1 = torch.tensor([1.0, 1.0], requires_grad=True)

In [0]:
v2 = torch.tensor([2.0, 2.0])

2개의 tensor를 생성했다.

v1은 연산을 위한 gradients를 require했고 v2는 그러지 않았다.

In [0]:
v_sum = v1 + v2

In [0]:
v_res = (v_sum*2).sum()

In [30]:
v_res

tensor(12., grad_fn=<SumBackward0>)

두 개의 vetor를 element-wise로 더했고([3, 3]), 더한 값을 2배로 곱해서([6, 6]) 요소들을 더했다.

그 결과 zero-dimension tensor의 값은 12가 되었다. 

tensor들의 attributes를 확인해보면 v1과 v2들이 leaf node들인지 그리고 v2를 제외한 모든 변수들이 gradients들을 require하고 있음을 알 수 있다.

In [31]:
v1.is_leaf, v2.is_leaf

(True, True)

In [32]:
v_sum.is_leaf, v_res.is_leaf

(False, False)

In [33]:
v1.requires_grad

True

In [34]:
v2.requires_grad

False

In [35]:
v_sum.requires_grad

True

In [36]:
v_res.requires_grad

True

이제 Pytorch에게 그래프의 gradients들을 계산 시켜본다.

In [0]:
v_res.backward()

In [38]:
v1.grad

tensor([2., 2.])

the backward function으로 그래프에 있는 다른 변수들도 고려하여 v_res variable에 대한 미분을 할 수 있었다.

다시 말해, 그래프의 다른 요소들로 인해 the v_res variable는 어떻게 변화하는가?

예제에서 보듯이,  v1의 gradient 값인 2는 v1이 1증가할 때마다 v_res는 2씩 증가한다는 것을 알 수 있다.

이전에 말했듯이 PyTorch는 requires_grad=True인 leaf tenor들에 대해서만 gradient를 계산할 수 있다. 그래서 v2의 gradient를 확인해보면 아무것도 나오지 않는다.

In [0]:
v2.grad

compution과 memory의 효율성 때문에 이렇게 따로 requires_grad=True로 한 변수들에 대해서만 계산해주는 것이다.

gradient descent optimization을 할 때, 모든 매트릭스 중간 연산들에 대해서 gradient를 알고 싶은 것이 아니다. 우리는 오직 모델의parameter(weights)의 loss gradient만 원한다.

물론 input data에 대해 gradient를 구하고 싶다면 requires_grad=True를 하면 된다. (it could be useful if you want to generate some adversarial examples to fool the existing NN or adjust pretrained word embeddings)

# NN building blocks

For example, the Linear class implements a feed-forward layer with optional bias:

In [0]:
import torch.nn as nn

In [0]:
l = nn.Linear(2, 5)

In [0]:
v = torch.FloatTensor([1, 2])

In [43]:
l(v)

tensor([ 0.3427, -0.2845, -2.2182,  0.0536, -0.0588], grad_fn=<AddBackward0>)

Here, we created a randomly initialized feed-forward layer, with two inputs and five outputs, and applied it to our float tensor.

## Sequential Class

The best way to demonstrate Sequential is through an example:

In [0]:
s = nn.Sequential(
nn.Linear(2, 5),
nn.ReLU(),
nn.Linear(5, 20),
nn.ReLU(),
nn.Linear(20, 10),
nn.Dropout(p=0.3),
nn.Softmax(dim=1))

In [45]:
s

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=20, bias=True)
  (3): ReLU()
  (4): Linear(in_features=20, out_features=10, bias=True)
  (5): Dropout(p=0.3)
  (6): Softmax()
)

Here, we defined a three-layer NN with softmax on output, 
applied along dimension 1 (dimension 0 is batch samples), ReLU nonlinearities and dropout.

Let's push something through it:

In [46]:
s(torch.FloatTensor([[1,2]]))

tensor([[0.0741, 0.0741, 0.1361, 0.0741, 0.1844, 0.0741, 0.0741, 0.0741, 0.1607,
         0.0741]], grad_fn=<SoftmaxBackward>)

So, our minibatch is one example successfully traversed through the network!

# Custom Layers


Let's look at how this can be done for our Sequential example from the previous section, but in a more generic and reusable way (full sample is Chaper03/01modules.py):

This is our module class that inherits nn.Module. In the constructor, we pass three parameters : 


*   the size of input
*   size of output
*   optional dropout probability



1.   call the parent's constructor to let it initializw itself.
2.   create an already familiar nn.Sequential with a bunch of layers and assign it to our class field named pip. 
3.   By assigning a Sequential instance to our field, we automatically resigter this module (nn.Sequential inherits from nn.Module as does everything in the nn package)
4.   To register, we don't need to call anything, we just assign our submodules to fields.

After the constructor finishes, all those fields will be registered automatically ( if you really want to , there is a function in nn.Module to register submodules):





In [0]:
class OurModule(nn.Module):
    def __init__(self, num_inputs, num_classes, dropout_prob=0.3):
      super(OurModule, self).__init__()
      self.pipe = nn.Sequential(
      nn.Linear(num_inputs, 5),
      nn.ReLU(),
      nn.Linear(5, 20),
      nn.ReLU(),
      nn.Linear(20, num_classes),
      nn.Dropout(p=dropout_prob),
      nn.Softmax()
      )
    def forward(self, x):
      return self.pipe(x)

Here, we override the forward function with our implementaion of data transformation. 

As our module is a very simple wrapper around other layers, we just need to ask them to transform the data.

Note that to apply a module to the data, you need to call the module as callable (that is, pretend that the module instance is a function and call it with the arguments) and not use the forward() function of the nn.Module class. 

This is because nn.Module overrides the __call__() method, which is being used when we treat an instance as callable.

This method does some nn.Module magic stuff and calls your forward() method.

If you call forward() directly, you'll intervene with the nn.Module duty, which can give you wrong results.


So, that's what we need to do define our own module.

Now, let's use it:


In [48]:
if __name__ == "__main__":
  net = OurModule(num_inputs=2, num_classes=3)
  v = torch.FloatTensor([[2,3]])
  out = net(v)
  print(net)
  print(out)

OurModule(
  (pipe): Sequential(
    (0): Linear(in_features=2, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
    (5): Dropout(p=0.3)
    (6): Softmax()
  )
)
tensor([[0.3283, 0.3869, 0.2849]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


We create our module, providing it with the desired number of inputs and outputs, then we create a tensor, wrapped into the Variable and ask our module to transform it, following the same convention of using it as callable. 

Then we print our network's structure (nn.Module overrides __str__() and __repr__()) to represent the inner structure in a nice way. 

The last thing we show is the result of the network's transformation.

The output of our code should look like this:

# Optimizers
Now, let's discuss the common blueprint of a training loop:

In [0]:
for batch_samples, batch_labels in iterate_batches(data, batch_size=32):
  #1
  batch_samples_t = torch.tensor(batch_samples)
  #2
  batch_labels_t = torch.tensor(batch_labels)
  #3
  out_t = net(batch_samples_t)
  #4
  loss_t = loss_function(out_t, batch_labels_t)
  #5
  loss_t.backward()
  #6
  optimizer.step()
  #7
  optimizer.sero_grad()
  #8

# *Plotting stuff
the full example code is in Chapter03/02_tensorboard.py

In [50]:
pip install tensorboardX

     |████████████████████████████████| 245kB 3.0MB/s 


In [0]:
import math
from tensorboardX import SummaryWriter

In [0]:
if __name__ == "__main__":
  writer = SummaryWriter()
  
  funcs = {"sin":math.sin, "cos":math.cos, "tan":math.tan}

We import the required packages, create a writer of data, and define functions that we're going to visualize.

By default, SummaryWriter will create a unique directory under the runs directory for every launch, to be able to compare different launches of training.

Names of the new directory include the current date and time, and hostname.

To override this, you can pass the log_dir argument to SummaryWriter.

you also can add a suffix to the name of the directory by passing a comment option, for example to capture different experiments' semanrics, such as dropout=0.3 or strong_regularisation

In [0]:
for angle in range(-360, 360):
  angle_rad = angle * math.pi / 180
  for name, fun in funcs.items():
    val = fun(angle_rad)
    writer.add_scalar(name, val, angle)
    writer.close()

Here, we loop over angle ranges in degrees, convert them into radians, and calculate our functions' values.

Every values is being added to the writer using the add_scalar function, which takes three argumetns : the name of the parameter, its valus, and the current iteration( which has to be an integer).

The last thing we need to do after the loop is to close the writer. 

Note that the writer does a periodical flush ( by default, every two minutes), so even in the case of a lengthy optimization process, you still will see your valus.

The result of running this will be zero output on the console, but you will see a new directory created inside the runs directory with a single file.

To look at the result, we need to start TensorBoard:



In [0]:
import math
from tensorboardX import SummaryWriter


if __name__ == "__main__":
    writer = SummaryWriter()

    funcs = {"sin": math.sin, "cos": math.cos, "tan": math.tan}

    for angle in range(-360, 360):
        angle_rad = angle * math.pi / 180
        for name, fun in funcs.items():
            val = fun(angle_rad)
            writer.add_scalar(name, val, angle)

    writer.close()

In [67]:
!tensorboard --logdir runs --host 9df31622c663

TensorBoard 1.13.1 at http://9df31622c663:6006 (Press CTRL+C to quit)
^C


[텐서보드 참고](https://zzsza.github.io/data/2018/08/30/google-colab/#tensorboard-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0)

In [69]:
LOG_DIR = 'drive/data/tb_logs'
	
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
	
import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
	  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))
	
get_ipython().system_raw('./ngrok http 6006 &')
	
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-05-27 02:04:19--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.49.195, 54.236.200.27, 52.73.94.166, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.49.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  68.9MB/s    in 0.2s    

2019-05-27 02:04:20 (68.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://027fafa8.ngrok.io


In [71]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

### input image dimensions
img_rows, img_cols = 28, 28

### the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

### convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 6s 95us/step - loss: 0.2714 - acc: 0.9169 - val_loss: 0.0589 - val_acc: 0.9814
Epoch 2/12
60000/60000 [==============================] - 5s 80us/step - loss: 0.0916 - acc: 0.9723 - val_loss: 0.0425 - val_acc: 0.9858
Epoch 3/12
60000/60000 [==============================] - 5s 76us/step - loss: 0.0681 - acc: 0.9792 - val_loss: 0.0360 - val_acc: 0.9878
Epoch 4/12
60000/60000 [==============================] - 5s 75us/step - loss: 0.0576 - acc: 0.9827 - val_loss: 0.0339 - val_acc: 0.9886
Epoch 5/12
60000/60000 [=====

KeyboardInterrupt: ignored

# Example - Gan on Atari images

So, let's get started.

The whole example code is in the file Chapter03/03_atari_gan.py

Here we'// look at only significant pieces of code, without the import section and constant declaration:


In [0]:
import gym
import gym.spaces


In [0]:
class InputWrapper(gym.ObservationWrapper):
    """
    Preprocessing of input numpy array:
    1. resize image into predefined size
    2. move color channel axis to a first place
    """
    def __init__(self, *args):
        super(InputWrapper, self).__init__(*args)
        assert isinstance(self.observation_space, gym.spaces.Box)
        old_space = self.observation_space
        self.observation_space = gym.spaces.Box(self.observation(old_space.low), self.observation(old_space.high),
                                                dtype=np.float32)

    def observation(self, observation):
        # resize image
        new_obs = cv2.resize(observation, (IMAGE_SIZE, IMAGE_SIZE))
        # transform (210, 160, 3) -> (3, 210, 160)
        new_obs = np.moveaxis(new_obs, 2, 0)
        return new_obs.astype(np.float32)

This class is wrapper around a Gym game, which includes several transformations:
 *   Resize input image from 210 * 160 (standard Atari resolution) to a square size 64 * 64
 *   Move color plane of the image from the last position to first, to meet the PyTorch convention of convolution layers that input a tensor with the shape of channels, height, and width
 *   Cast the image from bytes to float and rescale its values to a 0..1 range
 
 
 Then we define nn.Module classes:
 *   Discriminator
 *   Generator
 
 
 The first takes our scaled color image as input and, by applying five layers of convolutions, converts it into a single number, passed through a sigmoid nonlinearity.
 
 The output from Sigmoid is interpreted as the probability that Discriminator thinks our input image is from the real dataset.
 
 Generator takes as inout a vector of random numbers (latent vector) and using the "transposed convolution" operation (it is also known as deconvolution), converts this vector into a color image of the original resolution.
 
We will not look at those classses here as they are lengthy and not very relevant to our example.

You can find them in the complete example file.

As input, we'll use screenshots from several Atari games played simultaneously by a random agent.

Figure 6 is an example of what the input data looks like and it is generated by the following function:





In [0]:
def iterate_batches(envs, batch_size=BATCH_SIZE):
    batch = [e.reset() for e in envs]
    env_gen = iter(lambda: random.choice(envs), None)

    while True:
        e = next(env_gen)
        obs, reward, is_done, _ = e.step(e.action_space.sample())
        if np.mean(obs) > 0.01:
            batch.append(obs)
        if len(batch) == batch_size:
            # Normalising input between -1 to 1
            batch_np = np.array(batch, dtype=np.float32) * 2.0 / 255.0 - 1.0
            yield torch.tensor(batch_np)
            batch.clear()
        if is_done:
            e.reset()

This infinitely samples the environment from the provided array, random actions and remembers observations in the batch list. 

When the batch becomes of the required size, we covert it to a tensor and yield from the generatoe.

The check for the nonzero mean observation is required due to a bug in on e of the games to prevent the filckering of an image.

Now let's look at our main function, which prepares models and runs the training loop:



In [0]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--cuda", default=False, action='store_true', help="Enable cuda computation")
    args = parser.parse_args()

    device = torch.device("cuda" if args.cuda else "cpu")
    envs = [InputWrapper(gym.make(name)) for name in ('Breakout-v0', 'AirRaid-v0', 'Pong-v0')]
    input_shape = envs[0].observation_space.shape

Here, we process the command-line arguments (which could be only one optional arguments, --cuda, enabling GPU computation mode) and create our environment pool with a wrapper applied.

This environment array will be passed to the iterate_batches function to generate training data:


In [0]:
    writer = SummaryWriter()
    net_discr = Discriminator(input_shape=input_shape).to(device)
    net_gener = Generator(output_shape=input_shape).to(device)

    objective = nn.BCELoss()
    gen_optimizer = optim.Adam(params=net_gener.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    dis_optimizer = optim.Adam(params=net_discr.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))


In this piece, we create our classes:
*   a summary writer
*   both networks
*   a loss function
*   two optimizers

Why two?

It's because that's the way that GANs get trained: to train the discriminator, we need to show it both real and fake data samples with appropriate labels (1 for real, 0 for fake).

During this pass, we update only the discriminator's parameters.

After that, we pass both real and fake samples through the discriminator again, but this time the labels are 1s for all samples, and now we update only the generator's weights.

The second pass teaches the generator how to fool the discriminator and confuse real samples with the generated ones:


In [0]:
    gen_losses = []
    dis_losses = []
    iter_no = 0

    true_labels_v = torch.ones(BATCH_SIZE, dtype=torch.float32, device=device)
    fake_labels_v = torch.zeros(BATCH_SIZE, dtype=torch.float32, device=device)

Here, we define arrays, which will be used to accumulate losses, iterator counters, and variables with the True and Fake labels.

In [0]:
    for batch_v in iterate_batches(envs):
        # generate extra fake samples, input is 4D: batch, filters, x, y
        gen_input_v = torch.FloatTensor(BATCH_SIZE, LATENT_VECTOR_SIZE, 1, 1).normal_(0, 1).to(device)
        batch_v = batch_v.to(device)
        gen_output_v = net_gener(gen_input_v)

At the beginning of the training loop, we generate a random vector and pass it to the Generator network.

In [0]:
        # train discriminator
        dis_optimizer.zero_grad()
        dis_output_true_v = net_discr(batch_v)
        dis_output_fake_v = net_discr(gen_output_v.detach())
        dis_loss = objective(dis_output_true_v, true_labels_v) + objective(dis_output_fake_v, fake_labels_v)
        dis_loss.backward()
        dis_optimizer.step()
        dis_losses.append(dis_loss.item())

At first, we train the discriminator by applying it two times: to the true data samples in our batch and to the generated ones.

We need to call the *detach()* function on the generator's output to prevent gradients of this training pass from flowing into the generator (*detach()* is a method of tensor, which makes a copy of it without connection to the parent's operation).

In [0]:
        # train generator
        gen_optimizer.zero_grad()
        dis_output_v = net_discr(gen_output_v)
        gen_loss_v = objective(dis_output_v, true_labels_v)
        gen_loss_v.backward()
        gen_optimizer.step()
        gen_losses.append(gen_loss_v.item())

Now it's the generator's training time.

We pass the generator's output to the discriminator, but now we don't stop the gradients.

Instead, we apply the objective function with True labels.

It will push our generator in the direction where the samples that it generates make the discriminator confuse them with the real data.

That's all real training, and the next couple of lines report losses and feed image samples to TensorBoard:

In [0]:
        iter_no += 1
        if iter_no % REPORT_EVERY_ITER == 0:
            log.info("Iter %d: gen_loss=%.3e, dis_loss=%.3e", iter_no, np.mean(gen_losses), np.mean(dis_losses))
            writer.add_scalar("gen_loss", np.mean(gen_losses), iter_no)
            writer.add_scalar("dis_loss", np.mean(dis_losses), iter_no)

The training of this example is quite a lengthy process.

On a GTX 1080 GPU, 100 iterations take about 40 seconds. 

At the beginning, the generated images are completely random noise, but after 10k-20k iterations, the generator becomes more and more proficient at its job and the generated images become more and more simliar to the real game screenshots.

My experiments gave the following images after 40k-50k of training iterations (several hours on a GPU):